## Optimizing model Parameters

- 이제 모델과 데이터를 확보했으니 데이터에 대한 모델을 최적화하여 교육, 검증 및 테스트할 차례입니다. 모델을 훈련시키는 것은 반복적인 과정이다. 각 반복(에포크라고 함)에서 모델은 출력에 대해 추측하고, 추측(손실)의 오류를 계산하고, (이전 섹션에서 살펴본 바와 같이) 해당 매개변수에 대한 오차의 파생물을 수집하고, 기울기 강하를 사용하여 이러한 매개 변수를 최적화한다.m

#### Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

/opt/tljh/user/envs/pytorch_gpu/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1627336316785/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
train_dataloader=DataLoader(training_data, batch_size=64)
test_dataloader=DataLoader(test_data, batch_size=64)

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

#### Hyperparameters
- 하이퍼 모수는 모형 최적화 프로세스를 제어할 수 있는 조정 가능한 모수입니다. 서로 다른 초 매개 변수 값은 모델 교육 및 수렴 속도에 영향을 미칠 수 있습니다(초 매개 변수 조정에 대한 자세한 내용 참조).


In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#### Optimization Loop

- 하이퍼 파라미터를 설정하면 최적화 루프를 사용하여 모델을 교육하고 최적화할 수 있습니다. 최적화 루프의 각 반복을 epoch(신기원)이라고합니다.
- Train Loop - 교육 데이터 세트에 대해 반복하고 최적의 매개 변수로 수렴하려고 합니다.
- 검증/테스트 루프 - 테스트 데이터 세트에 대해 반복하여 모델 성능이 향상되고 있는지 확인합니다.
- 이제 교육 루프에서 사용되는 몇 가지 개념을 간략히 숙지하겠습니다. 계속해서 최적화 루프의 전체 구현을 확인하십시오.

#### Loss Function

- 일부 교육 데이터와 함께 제시될 때, 우리의 훈련되지 않은 네트워크는 정답을 제공하지 않을 가능성이 높다.
- 손실함수는 얻은 결과와 목표값이 다른 정도를 측정하는데, 우리가 훈련 중에 최소화하고 싶은 것이 손실함수입니다.
- 손실을 계산하기 위해 주어진 데이터 샘플의 입력을 사용하여 예측하고 실제 데이터 레이블 값과 비교한다.
- Common loss functions include nn.MSELoss (Mean Square Error) for regression tasks, and nn.NLLLoss (Negative Log Likelihood) for classification. nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss.
- 우리는 모델의 출력 로짓을 nn.CrossEntropyLoss에 전달하여 로짓을 정규화하고 예측 오차를 계산합니다.


In [14]:
# Initialize the Loss function
loss_fn = nn.CrossEntropyLoss()

#### Optimizer
- Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
- Call optimizer.zero_grad() to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
- Backpropagate the prediction loss with a call to loss.backward(). PyTorch deposits the gradients of the loss w.r.t. each parameter.
- Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.


#### Full Implementation


- 우리는 최적화 코드를 통해 루프되는 train_loop과 테스트 데이터에 대해 모델의 성능을 평가하는 test_loop을 정의한다.

In [16]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        #Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

- 손실 함수와 최적화 프로그램을 초기화하고 train_loop 및 test_loop에 전달합니다. 모델의 향상된 성능을 추적할 수 있는 epoch의 수를 얼마든지 늘릴 수 있습니다.

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
----------------------------------
loss: 2.300167 [    0/60000]
loss: 2.289372 [ 6400/60000]
loss: 2.266861 [12800/60000]
loss: 2.255954 [19200/60000]
loss: 2.247903 [25600/60000]
loss: 2.222174 [32000/60000]
loss: 2.220986 [38400/60000]
loss: 2.188217 [44800/60000]
loss: 2.186215 [51200/60000]
loss: 2.153893 [57600/60000]
Test Error: 
 Accuracy: 56.2%, Avg loss: 2.146412 

Epoch 2
----------------------------------
loss: 2.159698 [    0/60000]
loss: 2.150538 [ 6400/60000]
loss: 2.086768 [12800/60000]
loss: 2.098682 [19200/60000]
loss: 2.052067 [25600/60000]
loss: 1.992160 [32000/60000]
loss: 2.011655 [38400/60000]
loss: 1.928404 [44800/60000]
loss: 1.936493 [51200/60000]
loss: 1.861063 [57600/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.860088 

Epoch 3
----------------------------------
loss: 1.897582 [    0/60000]
loss: 1.871301 [ 6400/60000]
loss: 1.744350 [12800/60000]
loss: 1.782959 [19200/60000]
loss: 1.684476 [25600/60000]
loss: 1.631893 [32000/60000]
loss: 1.64672